In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Han Santteut Dotum'   #한글 사용 가능

import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = 50
plt.style.use('ggplot')

In [7]:
def grap_year(data):
    data = str(data)
    return int(data[:4])

def grap_month(data):
    data = str(data)
    return int(data[4:])

In [8]:
from sklearn.preprocessing import LabelEncoder

In [9]:
def status(feature):
    print('Processing', feature, ': Done')
    print(f'Shape after processing {train.shape}')  #가공된 데이터의 형태를 확인하기 위해서| 쓰는게 좋다.
    print('*' * 40)

In [10]:
def get_train():
    train = pd.read_csv('C:/Users/chowonjae/Desktop/jeju_data_ver1/201901-202003.csv')
    train['year'] = train['REG_YYMM'].apply(lambda x: grap_year(x))
    train['month'] = train['REG_YYMM'].apply(lambda x: grap_month(x))
    train["DATE"] = pd.to_datetime(train["REG_YYMM"],format='%Y%m')
    return train

In [11]:
def process_card_sido(encoders):
    global train
    encoder = LabelEncoder()
    encoder.fit(train["CARD_SIDO_NM"])
    train["CARD_SIDO_NM"] = encoder.transform(train["CARD_SIDO_NM"])
    encoders["CARD_SIDO_NM"] = encoder
    status("CARD_SIDO_NM")
    return train

In [12]:
def process_hom_sido(encoders):
    global train
    encoder = LabelEncoder()
    encoder.fit(train["HOM_SIDO_NM"])
    train["HOM_SIDO_NM"] = encoder.transform(train["HOM_SIDO_NM"])
    encoders["HOM_SIDO_NM"] = encoder
    status("HOM_SIDO_NM")
    return train

In [13]:
def process_std(encoders):
    global train
    encoder = LabelEncoder()
    encoder.fit(train["STD_CLSS_NM"])
    train["STD_CLSS_NM"] = encoder.transform(train["STD_CLSS_NM"])
    encoders["STD_CLSS_NM"] = encoder
    status("STD_CLSS_NM")
    return train

In [14]:
def process_FLC(encoders):
    global train
    encoder = LabelEncoder()
    encoder.fit(train["FLC"])
    train["FLC"] = encoder.transform(train["FLC"])
    encoders["FLC"] = encoder
    status("FLC")
    return train

In [15]:
def process_AGE(encoders):
    global train
    encoder = LabelEncoder()
    encoder.fit(train["AGE"])
    train["AGE"] = encoder.transform(train["AGE"])
    encoders["AGE"] = encoder
    status("AGE")
    return train

In [16]:
def process_Sex(encoders):
    global train
    encoder = LabelEncoder()
    encoder.fit(train["SEX_CDGO_CD"])
    train["SEX_CDGO_CD"] = encoder.transform(train["SEX_CDGO_CD"])
    encoders["SEX_CDGO_CD"] = encoder
    status("SEX")
    return train

In [17]:
def process_month(date):  #date list
    global train
    tmp_train = pd.DataFrame(columns = train.columns)
    for d in date:
        tmp = train[train["DATE"] == d]
        tmp_train = tmp_train.append(tmp)
        
    train = tmp_train
    status("DATE")
    return train

In [18]:
def dropper():
    global train
    target = train["AMT"]
    train.drop(["REG_YYMM","CARD_CCG_NM", "SEX_CTGO_CD","HOM_CCG_NM", "CSTMR_CNT", "CNT","AMT", "DATE"], axis = 1, inplace = True)
    status("Drop")
    return train, target

In [19]:
encoders = {}
train = get_train()
train = process_card_sido(encoders)
train = process_hom_sido(encoders)
train = process_std(encoders)
train = process_FLC(encoders)
train = process_AGE(encoders)
#train = process_Sex()
#train = process_month(["2019-01","2019-02","2019-03","2019-04","2020-01","2020-02","2020-03"])
train, target = dropper()

Processing CARD_SIDO_NM : Done
Shape after processing (24697792, 15)
****************************************
Processing HOM_SIDO_NM : Done
Shape after processing (24697792, 15)
****************************************
Processing STD_CLSS_NM : Done
Shape after processing (24697792, 15)
****************************************
Processing FLC : Done
Shape after processing (24697792, 15)
****************************************
Processing AGE : Done
Shape after processing (24697792, 15)
****************************************
Processing Drop : Done
Shape after processing (24697792, 7)
****************************************


In [20]:
train = train.reset_index(drop = True)
target = target.reset_index(drop = True)

In [21]:
train.head(10)

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,FLC,year,month
0,0,0,0,1,0,2019,1
1,0,0,0,2,1,2019,1
2,0,0,0,2,1,2019,1
3,0,0,0,3,2,2019,1
4,0,0,0,3,3,2019,1
5,0,0,0,3,2,2019,1
6,0,0,0,4,3,2019,1
7,0,0,0,4,3,2019,1
8,0,0,0,5,4,2019,1
9,0,0,0,5,4,2019,1


In [22]:
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
from sklearn.model_selection import KFold #for K-fold cross validation
from sklearn.model_selection import cross_val_score #score evaluation
from sklearn.model_selection import cross_val_predict #prediction

In [23]:
CARD_SIDO_NMs = train['CARD_SIDO_NM'].unique()
STD_CLSS_NMs  = train['STD_CLSS_NM'].unique()
HOM_SIDO_NMs  = train["HOM_SIDO_NM"].unique()
AGEs          = train['AGE'].unique()
FLCs          = train['FLC'].unique()
years         = [2020]
months        = [4, 7]

temp = []
for CARD_SIDO_NM in CARD_SIDO_NMs:
    for STD_CLSS_NM in STD_CLSS_NMs:
        for HOM_SIDO_NM in HOM_SIDO_NMs:
            for AGE in AGEs:
                for FLC in FLCs:
                    for year in years:
                        for month in months:
                            temp.append([CARD_SIDO_NM, STD_CLSS_NM, HOM_SIDO_NM, AGE, FLC, year, month])
temp = np.array(temp)
temp = pd.DataFrame(data=temp, columns=["CARD_SIDO_NM", "STD_CLSS_NM", "HOM_SIDO_NM","AGE", "FLC", "year", "month"])

In [24]:
temp.head()

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,FLC,year,month
0,0,0,0,1,0,2020,4
1,0,0,0,1,0,2020,7
2,0,0,0,1,1,2020,4
3,0,0,0,1,1,2020,7
4,0,0,0,1,2,2020,4


In [25]:
##log를 씌운다

target = np.log1p((list(target.values)))

In [22]:
### HyperOpt Parameter Tuning
from hyperopt import tpe
from hyperopt import STATUS_OK
from hyperopt import Trials
from hyperopt import hp
from hyperopt import fmin

In [27]:
N_FOLDS = 10
MAX_EVALS = 5

In [28]:
def objective(params, n_folds = N_FOLDS):
    """Objective function for Logistic Regression Hyperparameter Tuning"""

    # Perform n_fold cross validation with hyperparameters
    # Use early stopping and evaluate based on ROC AUC

    clf = RandomForestRegressor(**params,random_state=0,verbose =0)
    scores = cross_val_score(clf, train, target, cv = N_FOLDS,scoring='neg_mean_squared_log_error')

    # Extract the best score
    best_score = max(scores)

    # Loss must be minimized
    loss = 1 - best_score

    # Dictionary with information for evaluation
    return {'loss': loss, 'params': params, 'status': STATUS_OK}

In [29]:
space = {
    'max_depth': hp.choice('max_depth', [3,4,5,6,7,8,9]),
    'warm_start' : hp.choice('warm_start', [True, False]),
    'min_samples_split' : hp.choice("min_samples_split", [2,3,4,5,6,7,8]),
    'min_samples_leaf' : hp.choice("min_samples_leaf", [2,3,4,5,6,7,8,9,10]),
    'max_features' : hp.choice("max_features", ["auto", "sqrt", "log2"]),
    'min_impurity_decrease' : hp.uniform("min_impurity_decrease", 0,10),
    'n_estimators' : hp.choice('n_estimators', range(100, 1000, 100)),
    "n_jobs" : -1
}

In [30]:
# Algorithm
tpe_algorithm = tpe.suggest

# Trials object to track progress
bayes_trials = Trials()

# Optimize
best = fmin(fn = objective, space = space, algo = tpe.suggest, max_evals = MAX_EVALS, trials = bayes_trials)

100%|██████████████████████████████████████████████| 5/5 [4:08:35<00:00, 2983.19s/trial, best loss: 1.0149469373198838]


In [31]:
best

{'max_depth': 4,
 'max_features': 2,
 'min_impurity_decrease': 4.585812961200192,
 'min_samples_leaf': 1,
 'min_samples_split': 4,
 'n_estimators': 7,
 'warm_start': 1}

In [ ]:
##1.0149469458658045

In [33]:
rf = RandomForestRegressor(
    max_depth =  7,
    max_features = "log2",
    min_impurity_decrease = 4.585812961200192,
    min_samples_leaf = 3,
    min_samples_split = 6,
    n_estimators = 800,
    warm_start = False,
    n_jobs = -1)

In [39]:
rf.fit(train, target)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=7, max_features='log2', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=4.585812961200192,
                      min_impurity_split=None, min_samples_leaf=3,
                      min_samples_split=6, min_weight_fraction_leaf=0.0,
                      n_estimators=800, n_jobs=-1, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [40]:
pred = rf.predict(temp)
pred = np.expm1(pred)
temp['AMT'] = np.round(pred, 0)
temp['REG_YYMM'] = temp['year']*100 + temp['month']
temp = temp[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
temp = temp.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)

In [41]:
# 디코딩 
temp['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(temp['CARD_SIDO_NM'])
temp['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(temp['STD_CLSS_NM'])

In [42]:
temp.head()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,202004,강원,건강보조식품 소매업,10922205.0
1,202004,강원,골프장 운영업,10922205.0
2,202004,강원,과실 및 채소 소매업,10922205.0
3,202004,강원,관광 민예품 및 선물용품 소매업,10922205.0
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,10922205.0


In [44]:
# 제출 파일 만들기
submission = pd.read_csv('C:/Users/chowonjae/Desktop/jeju_data_ver1/submission.csv', index_col=0)
submission = submission.drop(['AMT'], axis=1)
submission = submission.merge(temp, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
submission.index.name = 'id'
submission.to_csv('credit_submission2.csv', encoding='utf-8-sig')
submission.head()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,10922205.0
1,202004,강원,골프장 운영업,10922205.0
2,202004,강원,과실 및 채소 소매업,10922205.0
3,202004,강원,관광 민예품 및 선물용품 소매업,10922205.0
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,10922205.0


In [63]:
from sklearn.model_selection import KFold
import lightgbm as lgb

# 각 모델에 대한 oof 정의
lgbm_oof_train = np.zeros((train.shape[0]))
lgbm_oof_test = np.zeros((temp.shape[0]))

n_splits = 5
kfold = KFold(shuffle = True, n_splits = n_splits)

for train_idx, val_idx in kfold.split(X = train, y = target):
    X_train, X_valid = train.iloc[train_idx], train.iloc[val_idx]
    y_train, y_valid = target[train_idx], target[val_idx]
    
    model = lgb.LGBMRegressor(n_estimators = 500, objective = "rmse")
    evals = [(X_train, y_train),(X_valid, y_valid)]
    model.fit(X_train, y_train, eval_set = evals, early_stopping_rounds=100, verbose=50)
    
    lgbm_valid_pred = model.predict(X_valid)
    lgbm_test_pred = model.predict(temp)
     
    # local_valid/local_test에 대한 예측
    lgbm_valid_pred = model.predict(X_valid)
    lgbm_test_pred = model.predict(temp)
        
    lgbm_oof_train[val_idx] = lgbm_valid_pred
    lgbm_oof_test += lgbm_test_pred/ n_splits #Fold한 결과 앙상블(평균)

Training until validation scores don't improve for 100 rounds
[50]	training's rmse: 1.56667	valid_1's rmse: 1.56622
[100]	training's rmse: 1.55752	valid_1's rmse: 1.55714
[150]	training's rmse: 1.55385	valid_1's rmse: 1.55347
[200]	training's rmse: 1.55119	valid_1's rmse: 1.55082
[250]	training's rmse: 1.54941	valid_1's rmse: 1.54906
[300]	training's rmse: 1.54792	valid_1's rmse: 1.54757
[350]	training's rmse: 1.54679	valid_1's rmse: 1.54646
[400]	training's rmse: 1.54594	valid_1's rmse: 1.54562
[450]	training's rmse: 1.54523	valid_1's rmse: 1.54491
[500]	training's rmse: 1.54437	valid_1's rmse: 1.54407
Did not meet early stopping. Best iteration is:
[500]	training's rmse: 1.54437	valid_1's rmse: 1.54407
Training until validation scores don't improve for 100 rounds
[50]	training's rmse: 1.56728	valid_1's rmse: 1.5672
[100]	training's rmse: 1.55912	valid_1's rmse: 1.55905
[150]	training's rmse: 1.55426	valid_1's rmse: 1.5542
[200]	training's rmse: 1.5518	valid_1's rmse: 1.55177
[250]	tr

In [65]:
from math import sqrt
from sklearn.metrics import mean_squared_error

print(f"<Light-GBM> OVERALL RMSE: {sqrt( mean_squared_error( target, lgbm_oof_train ))}")
lgbm_oof_test1 = lgbm_oof_test

<Light-GBM> OVERALL RMSE: 1.544434041079817


In [40]:
model.fit(train, target)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=500, n_jobs=-1, num_leaves=31, objective='rmse',
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [41]:
pred = model.predict(temp)
pred = np.expm1(pred)
temp['AMT'] = np.round(pred, 0)
temp['REG_YYMM'] = temp['year']*100 + temp['month']
temp = temp[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
temp = temp.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)

In [42]:
# 디코딩 
temp['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(temp['CARD_SIDO_NM'])
temp['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(temp['STD_CLSS_NM'])

In [43]:
temp.head()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,202004,강원,건강보조식품 소매업,95065897.0
1,202004,강원,골프장 운영업,221325546.0
2,202004,강원,과실 및 채소 소매업,54584425.0
3,202004,강원,관광 민예품 및 선물용품 소매업,25307506.0
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,25348647.0


In [44]:
# 제출 파일 만들기
submission = pd.read_csv('C:/Users/chowonjae/Desktop/jeju_data_ver1/submission.csv', index_col=0)
submission = submission.drop(['AMT'], axis=1)
submission = submission.merge(temp, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
submission.index.name = 'id'
submission.to_csv('credit_submission3.csv', encoding='utf-8-sig')
submission.head()

#lb : 6.9376806925

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,95065897.0
1,202004,강원,골프장 운영업,221325546.0
2,202004,강원,과실 및 채소 소매업,54584425.0
3,202004,강원,관광 민예품 및 선물용품 소매업,25307506.0
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,25348647.0


In [23]:
from sklearn.model_selection import GridSearchCV

params = {'boosting_type' :['gbdt'],
          'learning_rate' : [0.02, 0.1],
          'num_leaves' : [50, 70],
          'min_data_in_leaf' : [700, 1000],
          'max_depth' : [5, 7],
          'tree_learner' : ['data'],
          'bagging_freq' : [10],
          'bagging_fraction' : [0.7, 1],
          'objective' : ['rmse']
         }

model = lgb.LGBMRegressor(n_estimators = 200, objective = "rmse", verbose = 50, early_stopping_rounds= 50)   #속도 향상을 위해 estimator를 200으로 줄임
gridcv = GridSearchCV(model, param_grid=params, verbose = True, n_jobs = -1)
gridcv.fit(train, target)  #속도 향상을 위해 50 epoch동안 성능이 향상되지 않으면 fitting 종료   

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed: 13.6min
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed: 44.0min finished


ValueError: For early stopping, at least one dataset and eval metric is required for evaluation

lightgbm에서 'boosting_type'을 'dart'로 하게 되면 모델은 매 iteration 마다 tree를 drop한다. 따라서 만약 early_stopping_rounds= 50라고 하면 모델은 50 iteration 전에 best score를 기록했는데 현재상황은 50 iteration을 진행하면서 몇 개의 tree를 drop 한 상황이다. 따라서 모델은50 iteration 전의 상황을 알 수 없다. 따라서 'dart' 모드에서 early_stopping_rounds 변수를 사용할 수 없다.

In [24]:
print('GridSearchCV 최적 파라미터:', gridcv.best_params_)

GridSearchCV 최적 파라미터: {'bagging_fraction': 0.7, 'bagging_freq': 10, 'boosting_type': 'gbdt', 'learning_rate': 0.02, 'max_depth': 5, 'min_data_in_leaf': 700, 'num_leaves': 50, 'objective': 'rmse', 'tree_learner': 'data'}


In [81]:
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train, target, test_size = 0.2, shuffle = True, random_state=0)

def lgb_roc_eval(n_estimators, learning_rate, num_leaves, min_data_in_leaf, bagging_fraction,reg_alpha, reg_lambda):
    
    params = {
        "boosting_type" : 'gbdt',
        'objective' : 'rmse',
        "n_estimators": 500,
        "learning_rate": learning_rate,
        'num_leaves': int(round(num_leaves)),
        "min_data_in_leaf": int(round(min_data_in_leaf)), 
        'bagging_fraction': bagging_fraction,
        'bagging_freq' : 10,
        'max_depth': 5,
        'reg_alpha': reg_alpha,
        'reg_lambda': reg_lambda,
        'tree_learner' : 'data',
        'verbosity': -1
    }
    print("params:", params)
    lgb_model = LGBMRegressor(**params)
    lgb_model.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=30, eval_metric = 'rmse', verbose=100 )
    best_iter = lgb_model.best_iteration_
    print('best_iter:', best_iter)
    valid_pred = lgb_model.predict(X_test)
    rmse = mean_squared_error(y_test, valid_pred, squared = False)
    print("rmse:", rmse)
    return -rmse

In [82]:
bayes_params = {
    "learning_rate" : (0.0001, 0.02),
    "num_leaves" : (30, 60),
    "min_data_in_leaf" : (100, 700),
    'bagging_fraction': (0.1, 0.7),
    "reg_alpha" : (0.0, 10),
    "reg_lambda" : (0.0, 10),
}

In [77]:
from bayes_opt import BayesianOptimization

BO_lgb = BayesianOptimization(lgb_roc_eval, bayes_params, random_state=0)

In [78]:
BO_lgb.maximize(init_points=5, n_iter=10)

|   iter    |  target   | baggin... | learni... | min_da... | n_esti... | num_le... | reg_alpha | reg_la... |
-------------------------------------------------------------------------------------------------------------
params: {'boosting_type': 'gbdt', 'objective': 'rmse', 'n_estimators': 590, 'learning_rate': 0.014332268390811148, 'num_leaves': 43, 'min_data_in_leaf': 462, 'bagging_fraction': 0.4292881023563948, 'bagging_freq': 10, 'max_depth': 5, 'reg_alpha': 6.458941130666561, 'reg_lambda': 4.375872112626925, 'tree_learner': 'data', 'verbosity': -1}
Training until validation scores don't improve for 30 rounds
[100]	valid_0's rmse: 1.60631
[200]	valid_0's rmse: 1.56723
[300]	valid_0's rmse: 1.53587
[400]	valid_0's rmse: 1.51384
[500]	valid_0's rmse: 1.49906
Did not meet early stopping. Best iteration is:
[590]	valid_0's rmse: 1.49129
best_iter: 590
rmse: 1.491293233248071
|  1        | -1.491    |  0.4293   |  0.01433  |  461.7    |  590.4    |  42.71    |  6.459    |  4.376    |
pa

params: {'boosting_type': 'gbdt', 'objective': 'rmse', 'n_estimators': 982, 'learning_rate': 0.01210833009836096, 'num_leaves': 31, 'min_data_in_leaf': 467, 'bagging_fraction': 0.20689338199830232, 'bagging_freq': 10, 'max_depth': 5, 'reg_alpha': 9.766757853394502, 'reg_lambda': 9.470549502554867, 'tree_learner': 'data', 'verbosity': -1}
Training until validation scores don't improve for 30 rounds
[100]	valid_0's rmse: 1.61736
[200]	valid_0's rmse: 1.58091
[300]	valid_0's rmse: 1.54862
[400]	valid_0's rmse: 1.52573
[500]	valid_0's rmse: 1.51005
[600]	valid_0's rmse: 1.497
[700]	valid_0's rmse: 1.48914
[800]	valid_0's rmse: 1.48364
[900]	valid_0's rmse: 1.47854
Did not meet early stopping. Best iteration is:
[982]	valid_0's rmse: 1.47393
best_iter: 982
rmse: 1.4739291125799159
|  10       | -1.474    |  0.2069   |  0.01211  |  467.1    |  982.2    |  31.13    |  9.767    |  9.471    |
params: {'boosting_type': 'gbdt', 'objective': 'rmse', 'n_estimators': 422, 'learning_rate': 0.00539238

In [79]:
BO_lgb.res   
#BayesianOptimization 객체의 res 속성은 하이퍼 파라미터 튜닝을 하는 과정에서의 metric 값과 그때의 하이퍼 파라미터 값을 가지고 있음.

[{'target': -1.491293233248071,
  'params': {'bagging_fraction': 0.4292881023563948,
   'learning_rate': 0.014332268390811148,
   'min_data_in_leaf': 461.65802564298633,
   'n_estimators': 590.3948646972071,
   'num_leaves': 42.709643980167144,
   'reg_alpha': 6.458941130666561,
   'reg_lambda': 4.375872112626925}},
 {'target': -1.4596440758685783,
  'params': {'bagging_fraction': 0.6350638004692478,
   'learning_rate': 0.019276888933970483,
   'min_data_in_leaf': 330.06491129546663,
   'n_estimators': 812.5525342743981,
   'num_leaves': 45.866847592587135,
   'reg_alpha': 5.680445610939323,
   'reg_lambda': 9.25596638292661}},
 {'target': -1.6029512823859329,
  'params': {'bagging_fraction': 0.14262163491873217,
   'learning_rate': 0.0018338730640606602,
   'min_data_in_leaf': 112.13103846419543,
   'n_estimators': 849.3578609931442,
   'num_leaves': 53.34470252849552,
   'reg_alpha': 8.700121482468191,
   'reg_lambda': 9.78618342232764}},
 {'target': -1.5913317843555976,
  'params': 

In [80]:
BO_lgb.max

{'target': -1.4515869171914897,
 'params': {'bagging_fraction': 0.18287042053076047,
  'learning_rate': 0.017637281635239088,
  'min_data_in_leaf': 279.4157090061244,
  'n_estimators': 996.9926669702271,
  'num_leaves': 39.05758413091597,
  'reg_alpha': 1.017998582485382,
  'reg_lambda': 9.954748285188833}}

In [26]:
from lightgbm import LGBMRegressor 

max_params = {'bagging_fraction': 0.18287042053076047,
  'learning_rate': 0.017637281635239088,
  'min_data_in_leaf': 279.4157090061244,
  'n_estimators': 996.9926669702271,
  'num_leaves': 39.05758413091597,
  'reg_alpha': 1.017998582485382,
  'reg_lambda': 9.954748285188833}

max_params["n_estimators"] = int(round(max_params["n_estimators"]))
max_params["num_leaves"] = int(round(max_params["num_leaves"]))
max_params["min_data_in_leaf"] = int(round(max_params["min_data_in_leaf"]))
max_params["boosting_type"] = 'gbdt'
max_params["objective"] = 'rmse'
max_params['bagging_freq'] = 10
max_params['max_depth'] = 5
max_params['tree_learner'] = 'data'
max_params["verbosity"] = -1

lgbm_max = LGBMRegressor(**max_params)

lgbm_max.fit(train, target)

LGBMRegressor(bagging_fraction=0.18287042053076047, bagging_freq=10,
              boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.017637281635239088,
              max_depth=5, min_child_samples=20, min_child_weight=0.001,
              min_data_in_leaf=279, min_split_gain=0.0, n_estimators=997,
              n_jobs=-1, num_leaves=39, objective='rmse', random_state=None,
              reg_alpha=1.017998582485382, reg_lambda=9.954748285188833,
              silent=True, subsample=1.0, subsample_for_bin=200000,
              subsample_freq=0, tree_learner='data', verbosity=-1)

In [27]:
pred = lgbm_max.predict(temp)
pred = np.expm1(pred)
temp['AMT'] = np.round(pred, 0)
temp['REG_YYMM'] = temp['year']*100 + temp['month']
temp = temp[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
temp = temp.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)

In [28]:
# 디코딩 
temp['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(temp['CARD_SIDO_NM'])
temp['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(temp['STD_CLSS_NM'])

In [29]:
# 제출 파일 만들기
submission = pd.read_csv('C:/Users/chowonjae/Desktop/jeju_data_ver1/submission.csv', index_col=0)
submission = submission.drop(['AMT'], axis=1)
submission = submission.merge(temp, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
submission.index.name = 'id'
submission.to_csv('credit_submission4.csv', encoding='utf-8-sig')
submission.head()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,121384301.0
1,202004,강원,골프장 운영업,247125597.0
2,202004,강원,과실 및 채소 소매업,58446358.0
3,202004,강원,관광 민예품 및 선물용품 소매업,26980560.0
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,26980560.0


In [ ]:
max_params

In [43]:
#clear Ram 

def clearRAM(print_usage=False):
    import os
    from hurry.filesize import size
    import psutil
    import gc
 
  #--- clear ram
    gc.collect()
 
  #--- print usage
    if print_usage == True:
        process = psutil.Process(os.getpid())
 
        print(size(process.memory_info().rss))

clearRAM(True)

2G
